In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import math
import os


main_dir = '/kaggle/input/optiver-trading-at-the-close/'
#main_dir = ''
train = pd.read_csv(main_dir + 'train.csv')
train = train.loc[train['target'].notna()]

In [2]:
# 填补空值far_price，near_price，返回三个结果
def imputer(df):
    far_price_mean = df['far_price'].mean()
    near_price_mean = df['near_price'].mean()
    df['far_price'] = df['far_price'].fillna(far_price_mean)
    df['near_price'] = df['near_price'].fillna(near_price_mean)

    return df, far_price_mean, near_price_mean

# 补充缺失的行
def add_missing_data(df):
    all_stock_ids = set(range(200))
    all_missed_data_list = []

    # 将数据预先进行分组，以便我们可以快速访问每个time_id的相关数据
    grouped = df.groupby('time_id')

    for t, group in grouped:
        current_stock_ids = set(group['stock_id'].to_list())
        missed_stock_id = list(all_stock_ids - current_stock_ids)
        
        date_id = group['date_id'].iloc[-1]
        seconds_in_bucket = group['seconds_in_bucket'].iloc[-1]
        
        missed_stock_id_num = len(missed_stock_id)
        missed_date_id = [date_id] * missed_stock_id_num
        missed_seconds_in_bucket = [seconds_in_bucket] * missed_stock_id_num
        missed_time_id = [t] * missed_stock_id_num
        
        missed_data = pd.DataFrame({
            'stock_id': missed_stock_id,
            'date_id': missed_date_id,
            'seconds_in_bucket': missed_seconds_in_bucket,
            'time_id': missed_time_id
        })
        
        all_missed_data_list.append(missed_data)

    all_missed_data = pd.concat(all_missed_data_list, axis=0).reset_index(drop=True).astype(int)

    df = pd.concat([df, all_missed_data], axis=0)
    df = df.sort_values(by=['time_id', 'stock_id']).reset_index(drop=True)
    df = df.groupby('stock_id').apply(lambda x: x.fillna(method='bfill')).reset_index(drop=True)

    return df

In [3]:
train, far_price_mean, near_price_mean = imputer(train)
train = add_missing_data(train)
print('空值数：', train.isnull().sum().sum())

空值数： 0


In [4]:
def sizesum_and_pricestd(df):
    # 更新后增加10个特征
    price_ftrs = ['reference_price', 'far_price', 'near_price', 'bid_price', 'ask_price', 'wap'] # std
    size_ftrs = ['imbalance_size', 'matched_size', 'bid_size', 'ask_size'] # sum
    
    rolled = df[['stock_id'] + size_ftrs].groupby('stock_id').rolling(window=6, min_periods=1).sum()
    rolled = rolled.reset_index(level=0, drop=True)
    for col in size_ftrs:
        df[f'{col}_rolled_sum'] = rolled[col]

    rolled = df[['stock_id'] + price_ftrs].groupby('stock_id').rolling(window=6, min_periods=1).std().fillna(0)
    rolled = rolled.reset_index(level=0, drop=True)
    for col in price_ftrs:
        df[f'{col}_rolled_std'] = rolled[col]

    return df

In [5]:
train = sizesum_and_pricestd(train)

In [6]:
# 删除列表元素
def remove_element(input_list, drop_list):
    return [e for e in input_list if e not in drop_list]

no_feature_cols = ['date_id', 'row_id', 'time_id', 'target', 'currently_scored']

feature_cols = remove_element(train.columns, no_feature_cols)
target_col = 'target'

In [7]:
print('特征数：', len(feature_cols))

特征数： 23


In [8]:
# 标准化
avg = train[feature_cols].mean()
std = train[feature_cols].std()

train[feature_cols] = (train[feature_cols] - avg)/std

In [9]:
# 将数据转为float32数据类型
train = train.astype('float32')

seq_len = 6

# Grouping by time_id
grouped_by_time = train.groupby('stock_id')

def generate_data(grouped_by_time, seq_len):
    for _, group in grouped_by_time:
        # Sorting by stock_id to maintain consistency across images
        group_sorted = group.sort_values(by='time_id')
        
        features = group_sorted[feature_cols].values

        windows = []

        for t in range(0, seq_len - 1):
            copy_0 = np.stack([features[0]] * (seq_len - 1 - t))
            cut_0 = features[: t + 1]
            windows.append(np.vstack((copy_0, cut_0)))
            
        for t in range(0, features.shape[0] - seq_len + 1):
            windows.append(features[t: t+seq_len, :])
        
        # Convert list of windows to numpy array
        features_array = np.stack(windows)
        
        target = group_sorted['target'].values

        # Yield the result for this group to avoid storing all results in memory
        yield features_array, target

# Use generator to iterate over data
data_generator = generate_data(grouped_by_time, seq_len=seq_len)

# If you need to store results in arrays:
datas, labels = zip(*data_generator)
data = np.array(datas).reshape(-1, seq_len, len(feature_cols))
label = np.array(labels).reshape(-1,)

#del train, datas, labels, grouped_by_time

In [10]:
# print(feature_cols, len(feature_cols))
# print(test_cols, len(test_cols))
# print([c for c in test_cols if c not in feature_cols])

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset, random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 打印设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('检查设备：', device)


data = torch.tensor(data, dtype=torch.float32).to(device)
label = torch.tensor(label, dtype=torch.float32).to(device)

print('特征形状', data.shape)
print('标签形状', label.shape)

# 分开训练集和验证集
torch.manual_seed(42)

dataset = TensorDataset(data, label)

train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

batch_size = 4096

train_loader = DataLoader(train_dataset, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)

print('batch形状：', next(iter(train_loader))[0].shape)

检查设备： cpu
特征形状 torch.Size([5291000, 6, 23])
标签形状 torch.Size([5291000])
batch形状： torch.Size([4096, 6, 23])


In [12]:
class MyModel(nn.Module):
    def __init__(self, feature_num, d_model, nhead, num_layers):
        super(MyModel, self).__init__()
        self.embedding = nn.Linear(feature_num, d_model)
        self.tf1 = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(0.5)
        self.tf2 = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_layers, batch_first=True)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = self.tf1.encoder(x)
        x = x[:, -1, :]
        x = self.fc(x)
        x = self.dropout(x)
        x = self.tf2.encoder(x)
        x = self.decoder(x)

        return x

In [13]:
is_train = True
if is_train:
    input_size = data.shape[-1]

    n_epochs = 50
    lr = 1e-03

    # pre mae init
    pre_epoch_valid_mae = np.inf

    # 当前学习率下，mae两轮不下降就就将学习率减半
    patience_counter = 0

    model = MyModel(feature_num=input_size, d_model=64, nhead=8, num_layers=1).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    loss = nn.L1Loss().to(device)

    out_path = "model/"
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    best_mae = np.inf

    print(f'Train start...')
    for epoch in range(n_epochs):
        model.train()
        train_maes = []
        batch_num = len(train_loader)

        # 训练
        for X, y in train_loader:
            optimizer.zero_grad()
            outputs = model(X).squeeze()
            l = loss(outputs, y)
            l.backward()
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()
            mae = l.item()
            train_maes.append(mae)
        epoch_train_mae = np.mean(train_maes)
        print(f'Epoch [{epoch+1}/{n_epochs}] Training average MAE: {epoch_train_mae:.4f}')
        train_maes = []

        # 验证
        model.eval()
        with torch.no_grad():
            valid_maes = []
            for X_v, y_v in valid_loader:
                preds = model(X_v).squeeze()
                mae = torch.abs(preds - y_v).mean().item()
                valid_maes.append(mae)
            epoch_valid_mae = np.mean(valid_maes)
            print(f'Epoch [{epoch+1}/{n_epochs}] Validation average MAE: {epoch_valid_mae:.4f}')

            if epoch_valid_mae < best_mae:
                best_mae = epoch_valid_mae
                torch.save(model, os.path.join(out_path, f"model_epoch_{epoch+1}.pt"))

        # 前一轮mae必须小于当前mae，否则学习率减半
        if epoch_valid_mae - pre_epoch_valid_mae > 0:
            patience_counter += 1

            if patience_counter == 2:
                lr = lr * 0.5
                patience_counter = 0
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr  # 更新学习率
                    print(f'renew lr to {lr}')

        # 更新mae
        pre_epoch_valid_mae = epoch_valid_mae

        # 劈叉超过0.03或者学习率低于1e-7，停止训练
        if (epoch_valid_mae - epoch_train_mae > 0.03) or (lr <1e-7):
            print('Early stop now.')
            break

    print(f'Train over.')

In [14]:
class TestStack:
    # 添加time_id
    def __init__(self, window_size=6):
        self.window_size = window_size * 2
        self.stock_cache = []  # Dictionary to hold cache for each stock

    def test_stack(self, test, time_id):
        # Convert batch_data to DataFrame if it's a list of dicts
        if isinstance(test, list):
            test = pd.DataFrame(test)
            
        test['time_id'] = time_id
        
        # 单条数据添加
        self.stock_cache.append(test)
        
        if len(self.stock_cache) > self.window_size:
            # 如果当前数据超过n条，就截取最后n条，把之前的丢掉
            self.stock_cache = self.stock_cache[-self.window_size:]
            test = pd.concat(self.stock_cache, axis=0).reset_index(drop=True)
        else:
            # 初始化，如果还有收集到n条数据，就把当前的数据复制6次
            self.stock_cache = []
            for t in range(self.window_size): # [0, 1, 2, 3, 4, 5]
                test['time_id'] = t - self.window_size + 1 # [-5, -4, -3, -2, -1, 0]
                test_add = test.copy()
                self.stock_cache.append(test_add)
            test = pd.concat(self.stock_cache, axis=0).reset_index(drop=True).sort_values(by='time_id')
            
        return test.sort_values(['time_id', 'stock_id'])

test_cols = None
def df_to_seq(test, seq_len):
    grouped_by_stock = test.groupby('stock_id')
    datas = []

    for _, group in grouped_by_stock:
        group_sorted = group.sort_values(by='time_id')
        cols = remove_element(test.columns, no_feature_cols)
        
        features = group_sorted[cols].values # [12, 23]
        
        features = features[-seq_len:, ]
        datas.append(features)

    return np.stack(datas)

def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    return out

In [15]:
is_infer = False
if is_infer:
    model_idx = [38, 39, 40, 41, 46]
    models = []
    for midx in model_idx:
        models.append(torch.load(f"/kaggle/input/transformers/model_epoch_{midx}.pt", map_location=device))

In [16]:
is_pre_test = False
if is_pre_test:
    # 提交前测试
    test_df = pd.read_csv(main_dir + 'example_test_files/test.csv')
    #test_df = test_df.drop(columns=['target'])
    test_group = test_df.groupby(['time_id'])
    tdp = TestStack(window_size=seq_len)

    counter = 0
    for test in test_group:
        test = test[1]
        test = test.drop(columns=['time_id'])

        # zerosum准备
        volumes = test.loc[:,'bid_size'] + test.loc[:,'ask_size']

        # 填补空值
        test['far_price'] = test['far_price'].fillna(far_price_mean)
        test['near_price'] = test['near_price'].fillna(near_price_mean)

        # 数据叠加
        test_stack = tdp.test_stack(test, counter)

        # 特征工程
        test = sizesum_and_pricestd(test_stack)

        # 标准化
        test_cols = remove_element(test.columns, no_feature_cols)
        test[test_cols] = (test[test_cols] - avg)/std

        # 序列化
        test = df_to_seq(test, seq_len)
    #     print(test.shape)

        # 推理
        predictions = np.zeros((test.shape[0],))
        for model in models:
            test = torch.tensor(test, dtype=torch.float32).squeeze().to(device)
            predictions_tmp = model(test).squeeze().cpu()
            predictions_tmp = predictions_tmp.detach().numpy()
            predictions += predictions_tmp
        
        predictions /= len(models)
        # zero sum调整
        predictions = zero_sum(predictions, volumes)

        # print(predictions)

        counter += 1

In [17]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()
tdp = TestStack(window_size=seq_len)

counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    
    if test.currently_scored.iloc[0]== False:
            sample_prediction['target'] = 0
            env.predict(sample_prediction)
            counter += 1
            continue
            
    # zerosum准备
    volumes = test.loc[:,'bid_size'] + test.loc[:,'ask_size']
    
    # 填补空值
    test['far_price'] = test['far_price'].fillna(far_price_mean)
    test['near_price'] = test['near_price'].fillna(near_price_mean)
        
    # 数据叠加
    test_stack = tdp.test_stack(test, counter)

    # 特征工程
    test = sizesum_and_pricestd(test_stack)

    # 标准化
    test_cols = remove_element(test.columns, no_feature_cols)
    test[test_cols] = (test[test_cols] - avg)/std

    # 序列化
    test = df_to_seq(test, seq_len)

    # 推理
    predictions = np.zeros((test.shape[0],))
    for model in models:
        test = torch.tensor(test, dtype=torch.float32).squeeze().to(device)
        predictions_tmp = model(test).squeeze().cpu()
        predictions_tmp = predictions_tmp.detach().numpy()
        predictions += predictions_tmp
    predictions /= len(models)

    # zero sum调整
    predictions = zero_sum(predictions, volumes)
    
    #print(predictions)

    # tensor转换为numpy，给结果赋值
    sample_prediction['target'] = predictions.values
    
    # 提交结果
    env.predict(sample_prediction)
    
    # 计数
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
